In [154]:
%serialconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f9e9af0b9b0, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.



In [81]:
%sendtofile --source i2cmodule.py

Sent 2 lines (72 bytes).


In [82]:
print(0x77, (i2c.scan()))

119 [119]


In [50]:
D1, D2 = MS5611readD()
print(MS5611convert(D1, D2)-99000)



2567.805


In [144]:
%%writefile ms5611_funcs.py

from i2cmodule import i2c

import ustruct, time

# this can go down to 9 but then the variable heating effects in the sensor from such rapid readings 
# really muck things up and causes spikes
msreaddelay = 20  

C3, C4, C6, SENST1, OFFT1, Tref = 0,0,0,0,0,0
def setupms5611(lmsreaddelay=20):
    global msreaddelay
    msreaddelay = lmsreaddelay
    global C3, C4, C6, SENST1, OFFT1, Tref
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    C3 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    C4 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    C6 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    SENST1 =  C1*0x8000
    OFFT1 = C2*0x10000
    Tref = C5*0x100
    
def MS5611convert(D1, D2):
    dT = D2 - Tref
    TEMP = 2000 + dT*C6/0x00800000 
    OFF = OFFT1 + dT*C4/0x80 
    SENS = SENST1 + dT*C3 / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * D1 / 0x200000 - OFF) / 0x8000 

def MS5611readD():
    i2c.writeto(0x77, b'\x48')
    time.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    time.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    return MS5611convert(D1, D2)

# takes the library module as an argument to save importing it here
async def aMS5611read(uasyncio):
    i2c.writeto(0x77, b'\x48')
    await uasyncio.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    await uasyncio.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    return MS5611convert(D1, D2)



Writing ms5611_funcs.py



In [145]:
%sendtofile --source ms5611_funcs.py

Sent 67 lines (2070 bytes).


In [146]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [155]:
from ms5611_funcs import setupms5611, MS5611readD

In [212]:
setupms5611(30)

In [216]:
for i in range(5):
    print(MS5611readD())

101392.2
101391.7
101392.3
101392.2
101392.3


In [161]:
import time
fout = open("baro1.txt", "w")
for i in range(15000):
    fout.write("%d %d\n" % (time.ticks_ms(), int(MS5611readD()-100000)))
    if (i%500) == 0:
        print(i)
    #fout.flush()
fout.close()

0
.....500
....1000
.....1500
....2000
.....2500
....3000
.....3500
....4000
.....4500
....5000
.....5500
....6000
.....6500
....7000
.....7500
....8000
.....8500
....9000
.....9500
....10000
.....10500
....11000
.....11500
....12000
.....12500
....13000
.....13500
....14000
.....14500
....

In [217]:
%capture --quiet ms5611readings4.txt
import time
for i in range(500):
    print("%d %d" % (time.ticks_ms(), int(MS5611readD()-100000)))


500 lines captured

In [162]:
import os
print(os.stat("baro1.txt"))

(32768, 0, 0, 0, 0, 0, 195000, 0, 0, 0)


In [163]:
%capture --quiet ms5611readings5.txt
for l in open("baro1.txt"):
    print(l, end="")


15000 lines captured

In [195]:
urandom.seed(123456)
print(urandom.uniform(0, 1))


0.01227069


In [197]:
# taken from http://www.design.caltech.edu/erik/Misc/Gaussian.html
import urandom, math
def gaussgen(seed=0):
    if seed:
        urandom.seed(seed)
    while True:
        while True:
            x1 = urandom.uniform(-1, 1)
            x2 = urandom.uniform(-1, 1)
            w = x1**2 + x2**2
            if w < 1.0:
                break
        w = math.sqrt((-2*math.log(w)) / w)
        yield x1 * w
        yield x2 * w



In [175]:
import math

# least squares terse functions (done as a list)
# [x0, n, Sx, Sx2, Sy, Sy2, Sxy]
def setpt0(lq, x, y):
    lq[0] = x
    lq[1] = 1
    lq[2] = 0
    lq[3] = 0
    lq[4] = y
    lq[5] = y**2
    lq[6] = 0

def addpt(lq, x, y):
    x = x - lq[0]
    lq[1] += 1
    lq[2] += x
    lq[3] += x**2
    lq[4] += y
    lq[5] += y**2
    lq[6] += x*y

def mergelq(lq, llq):
    dx = llq[0] - lq[0]
    nl = llq[1]
    lq[1] += nl
    lq[2] += llq[2] + nl*dx
    lq[3] += llq[3] + 2*llq[2]*dx + nl*dx**2
    lq[4] += llq[4]
    lq[5] += llq[5]
    lq[6] += llq[6] + llq[4]*dx

def copylq(lq, llq):
    for i in range(7):
        lq[i] = llq[i]
    
def calcmc(lq):
    m = (lq[6] - lq[2]*lq[4]/lq[1])/(lq[3] - lq[2]**2/lq[1])
    c = lq[4]/lq[1] - m*lq[2]/lq[1]
    rsq = m**2*lq[3] + lq[1]*c**2 + lq[5] + 2*m*c*lq[2] - 2*m*lq[6] - 2*c*lq[4]
    r = math.sqrt(rsq/lq[1])
    return m, c, r


In [208]:
# least squares accumulating into buffers and outputting
# into thinned pieces

lqA = [0,0,0,0,0,0,0,0,0,0]  # accumulator (last 3 is m,c,r)
lqR = [0,0,0,0,0,0,0]
lqC = [0,0,0,0,0,0,0]

sigma = 1.6
sectionlength = 16
bcopyrtoa = False
def processpt(t, v):
    global bcopyrtoa
    if bcopyrtoa:  # deferred
        copylq(lqA, lqR)
        lqR[1] = 0
        bcopyrtoa = False
        
    if lqR[1] == 0:
        setpt0(lqR, t, v)
        return False
    addpt(lqR, t, v)
    if lqR[1] < sectionlength:
        return False
    if lqA[1] == 0:
        copylq(lqA, lqR)
        lqR[1] = 0
        return False
    
    copylq(lqC, lqA)
    mergelq(lqC, lqR)
    m, c, r = calcmc(lqC)
    if r < sigma:
        copylq(lqA, lqC)
        lqA[7] = m
        lqA[8] = c
        lqA[9] = r
        lqR[1] = 0
        return False
    bcopyrtoa = True  # defer
    return True

processpt(time.ticks_ms(), int(MS5611readD()-100000))



In [180]:
print(lqR)

[10755904, 1, 0, 0, 1525, 2325625, 0]


In [209]:
sigma = 1.6
sectionlength = 16
for i in range(600):
    if processpt(time.ticks_ms(), int(MS5611readD()-100000)):
        print(lqR)

.[12514865, 16, 5170, 2303962, 24408, 37234626, 7883410]
[12515557, 16, 5171, 2301207, 24366, 37106516, 7875993]
[12516247, 16, 5172, 2305600, 24317, 36957425, 7858594]
[12516939, 16, 5188, 2317342, 24272, 36820784, 7867517]
[12517632, 16, 5167, 2297841, 24308, 36929972, 7852977]
[12518322, 16, 5163, 2296723, 24363, 37097519, 7869535]
[12519011, 16, 5164, 2297670, 24439, 37329203, 7893225]
[12519702, 16, 5173, 2303785, 24475, 37439295, 7911890]
[12520393, 16, 5183, 2311813, 24462, 37399590, 7920903]
[12521084, 16, 5186, 2318208, 24467, 37414699, 7932897]
.[12521777, 16, 5167, 2299307, 24469, 37420997, 7904937]
[12522468, 16, 5182, 2311120, 24502, 37521868, 7935933]
[12523161, 16, 5193, 2320543, 24514, 37558546, 7955374]
[12523855, 16, 5164, 2299310, 24496, 37503760, 7912285]
[12524546, 16, 5163, 2293629, 24505, 37531077, 7907986]
[12525236, 16, 5187, 2317773, 24540, 37638454, 7953359]
[12525929, 16, 5151, 2289857, 24554, 37681208, 7902779]
[12526620, 16, 5173, 2303185, 24536, 37626128,

In [207]:
# simulated values (that we can verify on the PC by replicating the code)
def Dprocesspt(t, v):
    processpt(t, v)
def getsuccessivei():
    i = 10000
    while True:
        yield i
        i += 45
        
lqR[1] = lqA[1] = 0
sigma = 1.5
gg = gaussgen(12345)
gs = getsuccessivei()
gsigma = 1.4
for i in range(100):
    Dprocesspt(next(gs), int(100.2+next(gg)*gsigma))
for i in range(100):
    Dprocesspt(next(gs), int(100.2+30*i/100+next(gg)*gsigma))
for i in range(100):
    Dprocesspt(next(gs), int(130.2-60*i/100+next(gg)*gsigma))
print(lqA[1], lqA[0], lqA[7], lqA[8], lqA[9])
    


112 10000 0.0002533169 98.98341 1.35949
80 15040 0.00684248 103.05 1.248749
16 18640 0.00684248 103.05 1.248749
80 19360 -0.01336043 124.8232 1.329944
